# UPYA ACQUISITION MODEL

In [1]:
import pandas as pd
import numpy as numpy
import csv as csv
import datetime as date

path_to_import = 'clients.csv'

import_file = pd.read_csv(path_to_import, sep=',', engine='python', encoding='utf-8-sig')

headers = pd.read_csv(path_to_import, nrows=0, sep=',').columns.tolist()

clients = pd.read_csv(path_to_import, sep=',')

clients.columns = clients.columns.str[1:]

clients = clients[['lientNumber','category','profile.firstName','profile.lastName','contact.mobile','status','signingDate','recorder',
             'profile.gender','profile.state','profile.district','profile.address','profile.landmark','profile.city','profile.gps.latitude',
             'profile.gps.longitude','profile.languages']]

clients = clients.rename(columns = {'lientNumber': 'clientNumber'}, inplace = False)

clients['signingDate'] = pd.to_datetime(clients['signingDate'], format='%m/%d/%Y')

clients['signingDate'] = pd.to_datetime(clients['signingDate']).dt.normalize()

clients = clients[((clients['signingDate'] >= '11/13/2023') & (clients['signingDate'] <= '11/13/2023')) & (clients['status'] == 'Signed')]

In [2]:
contracts = pd.read_csv('contracts.csv', usecols = ['client.clientNumber','contractNumber','paygNumber','dealName','status','signingDate','client.contact.fullNumber','onboardingStatus','schedule.upfrontPayment','schedule.minPayment','schedule.upfrontDays','totalCost'], encoding='windows-1252')

contracts = contracts[(contracts['dealName'] != 'LCP Demo (Perpetuity)') & (contracts['contractNumber'] != 295593406 ) & (contracts['onboardingStatus'] == 'Signed')]

#contracts = contracts[(contracts['dealName'] != 'LCP Demo (Perpetuity)') & ((contracts['onboardingStatus'] == 'Signed') | (contracts['onboardingStatus'] == 'Approved'))]

contracts['signingDate'] = pd.to_datetime(contracts['signingDate'].str[0:10]).dt.normalize()

#contracts['signingDate'] = pd.to_datetime(contracts['signingDate'].str[0:19]).... This helps to keep the DateTime format intact

contracts['signingDate'] = pd.to_datetime(contracts['signingDate'], format='%Y/%m/%d')

#contract

In [3]:
contracts = contracts[((contracts['signingDate'] >= '2023/11/13') & (contracts['signingDate'] <= '2023/11/13'))]

In [4]:
join1 = pd.merge(clients, contracts, how='inner', left_on='clientNumber', right_on = 'client.clientNumber')

In [5]:
join1['initial_down_payment'] = join1['schedule.upfrontPayment'] - join1['schedule.minPayment']

In [6]:
agents = pd.read_csv('agents.csv', usecols = ['agentNumber','profile.firstName','profile.lastName','profile.gender','entity.name','contact.mobile','role','location.district'], encoding='windows-1252')

In [7]:
join2 = pd.merge(join1, agents, how='left', left_on='recorder', right_on = 'agentNumber')

In [8]:

join3 = join2[['clientNumber','contractNumber','onboardingStatus','signingDate_x','paygNumber','category','status_y','dealName','schedule.upfrontPayment','schedule.minPayment',
               'initial_down_payment','totalCost','schedule.upfrontDays','profile.firstName_x','profile.lastName_x','contact.mobile_x','profile.gender_x','profile.languages',
               'profile.address','profile.landmark','profile.city','profile.district','profile.state','profile.gps.latitude','profile.gps.longitude','recorder','profile.firstName_y',
               'profile.lastName_y','profile.gender_y','contact.mobile_y','entity.name','role','location.district']]

In [9]:

join3.columns = ['client_number','contract_number','onboarding_status','created_date','system_id','grid_category','status','tenure',
                'full_initial_payment','monthly_payment','initial_down_payment','total_cost','upfront_days','first_name','last_name',
                'customer_contact','customer_gender','customer_language','customer_address','customer_landmark','customer_city',
                'customer_district','customer_state','customer_latitude','customer_longitude','agent_id','agent_first_name',
                'agent_last_name','agent_gender','agent_contact','sales_mk','role','agent_district']



In [10]:
#join3.info()

tm_mapping = pd.read_excel('tm_mapping.xlsx')

upya_acq = pd.merge(join3, tm_mapping, how= 'left', on= 'agent_district')


In [11]:
upya_acq['monthly_payment'] = upya_acq['monthly_payment'].fillna(0)
upya_acq['initial_down_payment'] = upya_acq['initial_down_payment'].fillna(0)
upya_acq['upfront_days'] = upya_acq['upfront_days'].fillna(0)
upya_acq['full_initial_payment'] = upya_acq['full_initial_payment'].fillna(0)
upya_acq['customer_latitude'] = upya_acq['customer_latitude'].fillna(0)
upya_acq['customer_longitude'] = upya_acq['customer_longitude'].fillna(0)
upya_acq['customer_landmark'] = upya_acq['customer_landmark'].fillna('N/A')
upya_acq['customer_city'] = upya_acq['customer_city'].fillna('N/A')
upya_acq['customer_district'] = upya_acq['customer_district'].fillna('N/A')
upya_acq['grid_category'] = upya_acq['grid_category'].fillna('N/A')
upya_acq['sm'] = upya_acq['sm'].fillna('N/A')
upya_acq['agent_gender'] = upya_acq['agent_gender'].fillna('N/A')
upya_acq['tm_first_name'] = upya_acq['tm_first_name'].fillna('N/A')
upya_acq['sm_state'] = upya_acq['sm_state'].fillna('N/A')
upya_acq['tm_last_name'] = upya_acq['tm_last_name'].fillna('N/A')

In [12]:
#import mysql.connector as msql
#from mysql.connector import Error

#try:
    #conn = msql.connect(host='165.232.88.149', 
                           #database='lumosNG', user='wale', 
                           #password='Olawale47253!', port='3306')
    #if conn.is_connected():
        #cursor = conn.cursor()
        #cursor.execute("select database();")
        #record = cursor.fetchone()
        #print("You're connected to lumosNG database: ", record)
        #cursor.execute('DROP TABLE IF EXISTS lcp_base;')
        #cursor.execute("Delete from upya_lcp_base where created_date > '2023/06/30'")
        #print("Data deleted from 2023/07/01")
        #print('Activity Table Truncated....')
        #print('Re-Creating Activity Table....')
        #cursor.execute("CREATE TABLE lcp_base (contract VARCHAR(50) NULL,system_id BIGINT NULL,contract_id BIGINT NULL,panel_id VARCHAR(50) NULL,created_date DATE NULL,customer_name VARCHAR(100) NULL,price_book_name VARCHAR(100) NULL,product_name VARCHAR(50) NULL,subtype VARCHAR(10) NULL,initial_rate_plan VARCHAR(20) NULL,full_initial_payment FLOAT(10,2) NULL,initial_down_payment FLOAT(10,2) NULL,sales_channel VARCHAR(20) NULL,sales_sub_channel VARCHAR(50) NULL,lcp_name VARCHAR(70) NULL,phone_number BIGINT NULL,alternate_contact BIGINT NULL,guarantor1_contact VARCHAR(20) NULL,guarantor2_contact VARCHAR(20) NULL,billing_street VARCHAR(230) NULL,community VARCHAR(50) NULL,local_government VARCHAR(100) NULL,billing_city VARCHAR(100) NULL,billing_state VARCHAR(50) NULL,location_longitude FLOAT(10,5) NULL,location_latitude FLOAT(10,5) NULL,TSM VARCHAR(50) NULL,store_name VARCHAR(50) NULL,status VARCHAR(20) NULL) ENGINE = InnoDB DEFAULT CHARACTER SET = utf8mb4 COLLATE = utf8mb4_general_ci;")
        #print("inserting into UPYA_lcp_base....")
        #for i,row in upya_acq.iterrows():
            #sql = "INSERT INTO lumosNG.upya_lcp_base VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            #cursor.execute(sql, tuple(row))
            #chunksize = 20
            #print("Record inserted")
            # the connection is not autocommitted by default, so we must commit to save our changes
            #conn.commit()
#except Error as e:
    #print("Error while connecting to MySQL", e)
    
#print ("UPYA_lcp_base completely updated")

In [13]:
upya_acq.isnull().sum().sort_values(ascending = False)

client_number           0
customer_landmark       0
customer_district       0
customer_state          0
customer_latitude       0
customer_longitude      0
agent_id                0
agent_first_name        0
agent_last_name         0
agent_gender            0
agent_contact           0
sales_mk                0
role                    0
agent_district          0
tm_first_name           0
tm_last_name            0
sm                      0
customer_city           0
customer_address        0
contract_number         0
customer_language       0
onboarding_status       0
created_date            0
system_id               0
grid_category           0
status                  0
tenure                  0
full_initial_payment    0
monthly_payment         0
initial_down_payment    0
total_cost              0
upfront_days            0
first_name              0
last_name               0
customer_contact        0
customer_gender         0
sm_state                0
dtype: int64

In [14]:
upya_acq.to_excel("Nov'23_data.xlsx")